# Modelo de Classificação de Depressão em Áreas Rurais

- Dataset para o desenvolvemento pego na plataforma [Kaggle](https://www.kaggle.com/datasets/diegobabativa/depression).


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.linear_model import LinearRegression
# %matplotlib inline


# Lendo o Dataset

In [ ]:

URL = 'https://gist.githubusercontent.com/Ronildo22/a8983b7543c364534c75d6add6fc8413/raw/767bec26dbf90a2467380aba7d6f71a0820bc37e/depressionDataset'
df_depressao = pd.read_csv(URL)

df_depressao.shape


## Renomeando o cabeçalho do Dataframe

In [ ]:

rename_df = {
            'Survey_id': 'id_pesquisa',
            'Ville_id': 'id_cidade',
            'sex': 'genero',
            'Age': 'idade',
            'Married': 'casado',
            'Number_children': 'numero_filhos',
            'education_level': 'nivel_educacao',
            'total_members': 'membros_familia',
            'gained_asset': 'ativos_adquiridos',
            'durable_asset': 'ativos_duraveis',
            'save_asset': 'ativos_poupados',
            'living_expenses': 'despesas_moradia',
            'other_expenses': 'outras_despesas',
            'incoming_salary': 'renda_salario',
            'incoming_own_farm': 'renda_fazenda_propria',
            'incoming_business': 'renda_negocios',
            'incoming_no_business': 'renda_sem_negocio',
            'incoming_agricultural': 'renda_agricola',
            'farm_expenses': 'despesas_fazenda',
            'labor_primary': 'trabalho_principal',
            'lasting_investment': 'investimento_duradouro',
            'no_lasting_investmen': 'sem_investimento_duradouro',
            'depressed': 'depressao'
            }

df_depressao.rename(columns=rename_df, inplace=True)
df_depressao.columns


## Removendo Colunas Desnecessarias

In [ ]:

df_depressao.drop(['id_pesquisa', 'id_cidade'], axis=1, inplace=True)

df_depressao.columns


## Removendo Valores Vazios do Dataframe

In [ ]:
df_depressao = df_depressao.dropna()

df_depressao.count()

## Removendo Dados Enviesados do Dataframe

In [ ]:

filtro = (
    (df_depressao['casado'] != 0) &
    (df_depressao['genero'] != 0) &
    (df_depressao['renda_salario'] != 1) &
    (df_depressao['renda_fazenda_propria'] != 1) &
    (df_depressao['renda_negocios'] != 1) &
    (df_depressao['renda_sem_negocio'] != 1) &
    (df_depressao['trabalho_principal'] != 1) &
    (df_depressao['depressao'] == 0)
)

linhas_a_serem_removidas = df_depressao[filtro]
df_depressao.drop(linhas_a_serem_removidas.index, inplace=True)

# RESENTANDO O INDICE DO DATAFRAME
df_depressao.reset_index(drop=True, inplace=True)

# VALOR ABSOLUTO QUE CADA COLUNA TEM QUE TER 
valor_absoluto = 230

### Removendo vies na coluna 'casado'

In [ ]:
cl_casado = df_depressao['casado']

lista_index_casado = []

cl_casado_value_0 = cl_casado.value_counts()[0]
cl_casado_value_1 = cl_casado.value_counts()[1]
quant_vies_remover = (cl_casado_value_1 - cl_casado_value_0) - 1

cl_casado.reset_index(drop=True, inplace=True)

i = 0
for index, valor in enumerate(cl_casado):

    if valor == 1 and i <= quant_vies_remover:

        lista_index_casado.append(index)
        cl_casado.drop(index=[index], inplace=True)
        i += 1

valor_atual = cl_casado.count()

quatidade_remover = valor_atual - valor_absoluto
quatidade_remover = quatidade_remover / 2

linhas_a_serem_removidas = cl_casado[cl_casado == 0].head(int(quatidade_remover))
cl_casado.drop(linhas_a_serem_removidas.index, inplace=True)

linhas_a_serem_removidas = cl_casado[cl_casado == 1].head(int(quatidade_remover))
cl_casado.drop(linhas_a_serem_removidas.index, inplace=True)

print(cl_casado.value_counts())
print(cl_casado.count())


### Removendo vies na coluna 'genero'

In [ ]:
cl_genero = df_depressao['genero']

lista_index_genero = []

cl_genero_value_0 = cl_genero.value_counts()[0]
cl_genero_value_1 = cl_genero.value_counts()[1]
quant_vies_remover = (cl_genero_value_1 - cl_genero_value_0) - 1

i = 0
for index, valor in enumerate(cl_genero):

    if valor == 1 and i <= quant_vies_remover:

        lista_index_genero.append(index)
        cl_genero.drop(index=[index], inplace=True)
        i += 1

print(cl_genero.value_counts())
print(cl_genero.count())

### Removendo vies da coluna 'renda_salario'

In [ ]:

cl_renda_salario = df_depressao['renda_salario']

lista_index_renda_salario = []

cl_renda_salario_value_0 = cl_renda_salario.value_counts()[0]
cl_renda_salario_value_1 = cl_renda_salario.value_counts()[1]
quant_vies_remover = (cl_renda_salario_value_0 - cl_renda_salario_value_1) - 1

i = 0
for index, valor in enumerate(cl_renda_salario):

    if valor == 0 and i <= quant_vies_remover:

        lista_index_renda_salario.append(index)
        cl_renda_salario.drop(index=[index], inplace=True)
        i += 1

valor_atual = cl_renda_salario.count()

quatidade_remover = valor_atual - valor_absoluto
quatidade_remover = quatidade_remover / 2

linhas_a_serem_removidas = cl_renda_salario[cl_renda_salario == 0].head(int(quatidade_remover))
cl_renda_salario.drop(linhas_a_serem_removidas.index, inplace=True)

linhas_a_serem_removidas = cl_renda_salario[cl_renda_salario == 1].head(int(quatidade_remover))
cl_renda_salario.drop(linhas_a_serem_removidas.index, inplace=True)

print(cl_renda_salario.value_counts())
print(cl_renda_salario.count())


### Removendo vies na coluna 'renda_fazenda_propria'

In [ ]:

cl_renda_fazenda_propria = df_depressao['renda_fazenda_propria']

lista_index_renda_fazenda_propria = []

cl_renda_fazenda_propria_value_0 = cl_renda_fazenda_propria.value_counts()[0]
cl_renda_fazenda_propria_value_1 = cl_renda_fazenda_propria.value_counts()[1]

quant_vies_remover = (cl_renda_fazenda_propria_value_0 - cl_renda_fazenda_propria_value_1) - 1

i = 0
for index, valor in enumerate(cl_renda_fazenda_propria):

    if valor == 0 and i <= quant_vies_remover:

        lista_index_renda_fazenda_propria.append(index)
        cl_renda_fazenda_propria.drop(index=[index], inplace=True)
        i += 1

valor_atual = cl_renda_fazenda_propria.count()

quatidade_remover = valor_atual - valor_absoluto
quatidade_remover = quatidade_remover / 2

linhas_a_serem_removidas = cl_renda_fazenda_propria[cl_renda_fazenda_propria == 0].head(int(quatidade_remover))
cl_renda_fazenda_propria.drop(linhas_a_serem_removidas.index, inplace=True)

linhas_a_serem_removidas = cl_renda_fazenda_propria[cl_renda_fazenda_propria == 1].head(int(quatidade_remover))
cl_renda_fazenda_propria.drop(linhas_a_serem_removidas.index, inplace=True)

print(cl_renda_fazenda_propria.value_counts())
print(cl_renda_fazenda_propria.count())


### Removendo vies na coluna 'renda_negocios'

In [ ]:

cl_renda_negocios = df_depressao['renda_negocios']

lista_index_renda_negocios = []

cl_renda_negocios_value_0 = cl_renda_negocios.value_counts()[0]
cl_renda_negocios_value_1 = cl_renda_negocios.value_counts()[1]

quant_vies_remover = (cl_renda_negocios_value_0 - cl_renda_negocios_value_1) - 1

i = 0
for index, valor in enumerate(cl_renda_negocios):

    if valor == 0 and i <= quant_vies_remover:

        lista_index_renda_negocios.append(index)
        cl_renda_negocios.drop(index=[index], inplace=True)
        i += 1

valor_atual = cl_renda_negocios.count()

quatidade_remover = valor_atual - valor_absoluto
quatidade_remover = quatidade_remover / 2

linhas_a_serem_removidas = cl_renda_negocios[cl_renda_negocios == 0].head(int(quatidade_remover))
cl_renda_negocios.drop(linhas_a_serem_removidas.index, inplace=True)

linhas_a_serem_removidas = cl_renda_negocios[cl_renda_negocios == 1].head(int(quatidade_remover))
cl_renda_negocios.drop(linhas_a_serem_removidas.index, inplace=True)

print(cl_renda_negocios.value_counts())
print(cl_renda_negocios.count()) 

### Removendo vies da coluna 'renda_sem_negocio'

In [ ]:

cl_renda_sem_negocio = df_depressao['renda_sem_negocio']

lista_index_renda_sem_negocio = []

cl_renda_sem_negocio_value_0 = cl_renda_sem_negocio.value_counts()[0]
cl_renda_sem_negocio_value_1 = cl_renda_sem_negocio.value_counts()[1]

quant_vies_remover = (cl_renda_sem_negocio_value_0 - cl_renda_sem_negocio_value_1) - 1

i = 0
for index, valor in enumerate(cl_renda_sem_negocio):

    if valor == 0 and i <= quant_vies_remover:

        lista_index_renda_sem_negocio.append(index)
        cl_renda_sem_negocio.drop(index=[index], inplace=True)
        i += 1

print(cl_renda_sem_negocio.value_counts())

valor_atual = cl_renda_sem_negocio.count()

quatidade_remover = valor_atual - valor_absoluto
quatidade_remover = quatidade_remover / 2

linhas_a_serem_removidas = cl_renda_sem_negocio[cl_renda_sem_negocio == 0].head(int(quatidade_remover))
cl_renda_sem_negocio.drop(linhas_a_serem_removidas.index, inplace=True)

linhas_a_serem_removidas = cl_renda_sem_negocio[cl_renda_sem_negocio == 1].head(int(quatidade_remover))
cl_renda_sem_negocio.drop(linhas_a_serem_removidas.index, inplace=True)

print(cl_renda_sem_negocio.value_counts())
print(cl_renda_sem_negocio.count())

### Removendo vies na coluna 'trabalho_principal'

In [ ]:

cl_trabalho_principal = df_depressao['trabalho_principal']

lista_index_trabalho_principal = []

cl_trabalho_principal_value_0 = cl_trabalho_principal.value_counts()[0]
cl_trabalho_principal_value_1 = cl_trabalho_principal.value_counts()[1]

quant_vies_remover = (cl_trabalho_principal_value_0 - cl_trabalho_principal_value_1) - 1

i = 0
for index, valor in enumerate(cl_trabalho_principal):

    if valor == 0 and i <= quant_vies_remover:

        lista_index_trabalho_principal.append(index)
        cl_trabalho_principal.drop(index=[index], inplace=True)
        i += 1

print(cl_trabalho_principal.value_counts())

valor_atual = cl_trabalho_principal.count()

quatidade_remover = valor_atual - valor_absoluto
quatidade_remover = quatidade_remover / 2

linhas_a_serem_removidas = cl_trabalho_principal[cl_trabalho_principal == 0].head(int(quatidade_remover))
cl_trabalho_principal.drop(linhas_a_serem_removidas.index, inplace=True)

linhas_a_serem_removidas = cl_trabalho_principal[cl_trabalho_principal == 1].head(int(quatidade_remover))
cl_trabalho_principal.drop(linhas_a_serem_removidas.index, inplace=True)

print(cl_trabalho_principal.value_counts())
print(cl_trabalho_principal.count())


### Removendo vies na coluna 'depressao'

In [ ]:

cl_depressao = df_depressao['depressao']

lista_index_depressao = []

cl_depressao_value_0 = cl_depressao.value_counts()[0]
cl_depressao_value_1 = cl_depressao.value_counts()[1]

quant_vies_remover = (cl_depressao_value_0 - cl_depressao_value_1) - 1

i = 0
for index, valor in enumerate(cl_depressao):

    if valor == 0 and i <= quant_vies_remover:

        lista_index_depressao.append(index)
        cl_depressao.drop(index=[index], inplace=True)
        i += 1

print(cl_depressao.value_counts())

valor_atual = cl_depressao.count()

quatidade_remover = valor_atual - valor_absoluto
quatidade_remover = quatidade_remover / 2

linhas_a_serem_removidas = cl_depressao[cl_depressao == 0].head(int(quatidade_remover))
cl_depressao.drop(linhas_a_serem_removidas.index, inplace=True)

linhas_a_serem_removidas = cl_depressao[cl_depressao == 1].head(int(quatidade_remover))
cl_depressao.drop(linhas_a_serem_removidas.index, inplace=True)

print(cl_depressao.value_counts())
print(cl_depressao.count())


### Adicionando as mudanças no DataFrame

In [ ]:

df_depressao.drop(columns=['casado', 'genero', 'renda_salario', 'renda_fazenda_propria', 'renda_negocios',
                           'renda_sem_negocio', 'trabalho_principal', 'depressao'], inplace=True)

cl_casado = cl_casado.dropna()
cl_casado.reset_index(drop=True, inplace=True)

cl_genero = cl_genero.dropna()
cl_genero.reset_index(drop=True, inplace=True)

cl_renda_salario = cl_renda_salario.dropna()
cl_renda_salario.reset_index(drop=True, inplace=True)

cl_renda_fazenda_propria = cl_renda_fazenda_propria.dropna()
cl_renda_fazenda_propria.reset_index(drop=True, inplace=True)

cl_renda_negocios = cl_renda_negocios.dropna()
cl_renda_negocios.reset_index(drop=True, inplace=True)

cl_renda_sem_negocio = cl_renda_sem_negocio.dropna()
cl_renda_sem_negocio.reset_index(drop=True, inplace=True)

cl_trabalho_principal = cl_trabalho_principal.dropna()
cl_trabalho_principal.reset_index(drop=True, inplace=True)

cl_depressao = cl_depressao.dropna()
cl_depressao.reset_index(drop=True, inplace=True)

df_depressao['casado'] = cl_casado
df_depressao['genero'] = cl_genero
df_depressao['renda_salario'] = cl_renda_salario
df_depressao['renda_fazenda_propria'] = cl_renda_fazenda_propria
df_depressao['renda_negocios'] = cl_renda_negocios
df_depressao['renda_sem_negocio'] = cl_renda_sem_negocio
df_depressao['trabalho_principal'] = cl_trabalho_principal
df_depressao['depressao'] = cl_depressao

df_depressao = df_depressao.dropna()

print(df_depressao.count())

print(df_depressao['casado'].value_counts())
print(df_depressao['genero'].value_counts())
print(df_depressao['renda_salario'].value_counts())
print(df_depressao['renda_fazenda_propria'].value_counts())
print(df_depressao['renda_negocios'].value_counts())
print(df_depressao['renda_sem_negocio'].value_counts())
print(df_depressao['trabalho_principal'].value_counts())
print(df_depressao['depressao'].value_counts())


# Preparando Dados

## Separando o rotulo y dos dados x

In [ ]:

df_depressao.head()

y = df_depressao['depressao']

df_depressao.drop(['depressao'], axis=1, inplace=True)



y = np.array(y)

# Deixando a feature x em uma faixa inicial de -1 a 1

In [ ]:

df_depressao

scaler = StandardScaler()
x = scaler.fit_transform(df_depressao)


# Treinando o modelo

## Metricas do modelo com seed = 0

In [ ]:
seed = 0

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y, random_state=seed)

# Criando o modelo de regressão linear
model = LinearRegression()

# Treinando o modelo com os dados de treino
model.fit(x_train, y_train)

# Fazendo previsões com os dados de teste
pred = model.predict(x_test)


threshold = 0.5
pred_binary = np.where(pred >= threshold, 1, 0)  # Aplicar o limiar para transformar em binário

# Calculando e exibindo as métricas após a binarização das previsões
accuracy = accuracy_score(y_test, pred_binary)
precision = precision_score(y_test, pred_binary)
recall = recall_score(y_test, pred_binary)
auc = roc_auc_score(y_test, pred)  # AUC usando as previsões contínuas

print(f'Acurácia: {accuracy:.2f}')
print(f'Precisão: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'AUC: {auc:.2f}')

# Curva ROC após a binarização das previsões
fpr, tpr, thresholds = roc_curve(y_test, pred)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % auc)
plt.plot([0, 1], [0, 1], 'k--')  # Linha diagonal
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()


# scaler = MinMaxScaler(feature_range=(0, 1))
# pred_scaled = scaler.fit_transform(pred.reshape(-1, 1))

# print(pred_scaled)

# print(y_test)


# acuracia = accuracy_score(y_test, pred_scaled)


# recall = recall_score(y_test, pred)
# recall = recall * 100

# precisao = precision_score(y_test, pred)
# precisao = precisao * 100

# f1 = f1_score(y_test, pred)
# f1 = f1 * 100

# mae = mean_absolute_error(y_test, pred)
# mae = mae * 100

# mse = mean_squared_error(y_test, pred)
# mse = mse * 100

# rmse = np.sqrt(mse)
# rmse 


# print('========== Metricas do modelo ==========')
# print(f'Acuracia: {acuracia:.0f}%')
# print(f'Recall: {recall:.0f}%')
# print(f'Precisao: {precisao:.2f}%')
# print(f'F1-score: {f1:.2f}%')
# print(f"Erro Absoluto Médio (MAE): {mae:.0f}%",)
# print(f"Erro Quadrático Médio (MSE): {mse:.0f}%")
# print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse:.2f}")

# # Plotando os dados e a linha de regressão
# plt.scatter(x, y, color='blue', label='Dados reais')
# plt.plot(x, model.predict(x), color='red', label='Regressão Linear')
# plt.xlabel('Variável independente')
# plt.ylabel('Variável dependente')
# plt.legend()
# plt.title('Regressão Linear em Python')
# plt.show()